In [45]:
# methods = [
#     'lime_20', # pert
#     'shap_20', # pert
#     'intgrad', # grad
#     'gradcam', # grad
#     'fullgrad', # grad
#     'rise_20', # pert
#     'archipelago', # pert?
#     'mfaba', # attak with grad
#     'agi', # attack with grad
#     'ampe', # attack
#     'bcos', # 
#     'xdnn', # gradient x input
#     'bagnet', # by construction
#     'attn', # by construction
#     'sop', # by construction
# ]

name_mapping = {
    'lime': {'name': 'LIME', 'category': 'Post Hoc'},
    'shap': {'name': 'SHAP', 'category': 'Post Hoc'},
    'rise': {'name': 'RISE', 'category': 'Post Hoc'},
    'lime_20': {'name': 'LIME', 'category': 'Post Hoc'}, # pert
    'shap_20': {'name': 'SHAP', 'category': 'Post Hoc'}, # pert
    'intgrad': {'name': 'IG', 'category': 'Post Hoc'}, # grad
    'gradcam': {'name': 'GC', 'category': 'Post Hoc'}, # grad
    'fullgrad': {'name': 'FG', 'category': 'Post Hoc'}, # grad
    'rise_20': {'name': 'RISE', 'category': 'Post Hoc'}, # pert
    'archipelago': {'name': 'Archi.', 'category': 'Post Hoc'}, # pert?
    'mfaba': {'name': 'MFABA', 'category': 'Post Hoc'}, # attak with grad
    'agi': {'name': 'AGI', 'category': 'Post Hoc'}, # attack with grad
    'ampe': {'name': 'AMPE', 'category': 'Post Hoc'}, # attack
    'bcos': {'name': 'BCos', 'category': 'Post Hoc'}, # 
    'xdnn': {'name': 'XDNN', 'category': 'Post Hoc'}, # gradient x input
    'bagnet': {'name': 'BagNet', 'category': 'Faithful'}, # by construction
    'attn': {'name': 'FRESH', 'category': 'Faithful'}, # by construction
    'sop': {'name': 'SOP', 'category': 'Faithful'}, # by construction
}

In [4]:
metrics = {
    'fid': 'fids_dict.pt',
    'ins': 'inss_dict.pt',
    'del': 'dels_dict.pt'
}

In [12]:
import random
import numpy as np

def bootstrap(data_list, num_bootstrap=4, num_samples=None, seed=0):
    if num_samples is None:
        num_samples = len(data_list)
    means = []
    random.seed(seed)
    for i in range(num_bootstrap):
        exp_idxs = random.choices(list(range(num_samples)), k=num_samples)
        means.append(np.mean([data_list[di] for di in exp_idxs]))
    return np.std(means)

In [47]:
import torch
from collections import defaultdict
import numpy as np

data = defaultdict(dict)

for metric in metrics:
    metric_data = torch.load(metrics[metric])
    for k in metric_data:
        # print(metric_data[k])
        metric_data_k = metric_data[k]
        if isinstance(metric_data_k, torch.Tensor):
            metric_data_k = metric_data_k.tolist()
        metric_mean = np.mean(metric_data_k)
        metric_std = bootstrap(metric_data_k)
        
        data[name_mapping[k]['name']][metric] = {
            'mean': metric_mean,
            'std': metric_std
        }

In [48]:
data['SOP']['fid'] = {'mean': 0, 'std': 0}
data['FRESH']['fid'] = {'mean': 0, 'std': 0}

In [49]:
import numpy as np

def format_value(value, std, bold=False, italic=False):
    if value is None or np.isnan(value):
        return '-'
    formatted = f"{value:.3f} $\\pm$ {std:.3f}"
    if bold:
        formatted = f"\\textbf{{{formatted}}}"
    elif italic:
        formatted = f"\\textit{{{formatted}}}"
    return formatted

def print_table(data, name_mapping):
    print("\\begin{tabularx}{\\textwidth}{c|c|*{3}{>{\\centering\\arraybackslash}X}}")
    print("\\toprule")
    print("\\multirow{2}{*}{Category} & \\multirow{2}{*}{Method} & \\multicolumn{3}{c}{\\textbf{ImageNet}} \\\\")
    print("& & Fid.$\\downarrow$ & Ins.$\\uparrow$ & Del.$\\downarrow$ \\\\")
    print("\\midrule")

    categories = ['Post Hoc', 'Faithful']
    metrics = ['fid', 'ins', 'del']

    all_values = []
    for category in categories:
        methods = [v['name'] for k, v in name_mapping.items() if v['category'] == category]
        for method in methods:
            values = []
            for metric in metrics:
                if method in data and metric in data[method] and data[method][metric] is not None:
                    mean = round(data[method][metric]['mean'], 6)
                    std = round(data[method][metric]['std'], 6)
                    values.append((mean, std))
                else:
                    values.append(None)
            all_values.append((category, method, values))

    # Determine best and second-best values for each column
    best_values = [min((v[0] for _, _, row in all_values for v in [row[0]] if v is not None), default=None),
                   max((v[0] for _, _, row in all_values for v in [row[1]] if v is not None), default=None),
                   min((v[0] for _, _, row in all_values for v in [row[2]] if v is not None), default=None)]
    
    second_best_values = [sorted(set(v[0] for _, _, row in all_values for v in [row[0]] if v is not None))[1] if len(set(v[0] for _, _, row in all_values for v in [row[0]] if v is not None)) > 1 else None,
                          sorted(set(v[0] for _, _, row in all_values for v in [row[1]] if v is not None), reverse=True)[1] if len(set(v[0] for _, _, row in all_values for v in [row[1]] if v is not None)) > 1 else None,
                          sorted(set(v[0] for _, _, row in all_values for v in [row[2]] if v is not None))[1] if len(set(v[0] for _, _, row in all_values for v in [row[2]] if v is not None)) > 1 else None]

    current_category = None
    for category, method, values in all_values:
        if category != current_category:
            if current_category is not None:
                print("\\midrule")
            print(f"\\multirow{{{len([m for c, m, _ in all_values if c == category])}}}{{*}}{{{category}}} & ", end="")
            current_category = category
        else:
            print("& ", end="")

        formatted_values = []
        for i, value in enumerate(values):
            if value is not None:
                is_best = value[0] == best_values[i]
                is_second_best = value[0] == second_best_values[i]
                formatted_values.append(format_value(value[0], value[1], bold=is_best, italic=is_second_best))
            else:
                formatted_values.append('-')

        print(f"{method} & {' & '.join(formatted_values)} \\\\")

    print("\\bottomrule")
    print("\\end{tabularx}")

# Your data dictionary and name_mapping remain the same
# ...

print_table(data, name_mapping)

\begin{tabularx}{\textwidth}{c|c|*{3}{>{\centering\arraybackslash}X}}
\toprule
\multirow{2}{*}{Category} & \multirow{2}{*}{Method} & \multicolumn{3}{c}{\textbf{ImageNet}} \\
& & Fid.$\downarrow$ & Ins.$\uparrow$ & Del.$\downarrow$ \\
\midrule
\multirow{15}{*}{Post Hoc} & LIME & 3.866 $\pm$ 0.244 & 0.859 $\pm$ 0.005 & 0.476 $\pm$ 0.004 \\
& SHAP & 0.015 $\pm$ 0.006 & \textit{0.878 $\pm$ 0.007} & 0.421 $\pm$ 0.008 \\
& RISE & 0.884 $\pm$ 0.533 & 0.635 $\pm$ 0.007 & 0.708 $\pm$ 0.003 \\
& LIME & 3.866 $\pm$ 0.244 & 0.859 $\pm$ 0.005 & 0.476 $\pm$ 0.004 \\
& SHAP & 0.015 $\pm$ 0.006 & \textit{0.878 $\pm$ 0.007} & 0.421 $\pm$ 0.008 \\
& IG & 7.161 $\pm$ 0.212 & 0.661 $\pm$ 0.006 & 0.664 $\pm$ 0.008 \\
& GC & 10.406 $\pm$ 1.098 & 0.817 $\pm$ 0.007 & 0.416 $\pm$ 0.007 \\
& FG & 13.567 $\pm$ 0.158 & 0.805 $\pm$ 0.006 & 0.430 $\pm$ 0.004 \\
& RISE & 0.884 $\pm$ 0.533 & 0.635 $\pm$ 0.007 & 0.708 $\pm$ 0.003 \\
& Archi. & 10.850 $\pm$ 0.354 & 0.719 $\pm$ 0.004 & 0.548 $\pm$ 0.004 \\
& MFABA & 6.6